In [1]:
import numpy as np
import pandas as pd

In [2]:
data = {
    "outlook" : ['sunny','sunny','overcast','rainy','rainy','rainy','overcast','sunny','sunny','rainy','sunny','overcast','overcast','rainy'],
    "temp" : ['hot','hot','hot','mild','cool','cool','cool','mild','cool','mild','mild','mild','hot','mild'],
    'humidity' : ['high','high','high','high','normal','normal','normal','high','normal','normal','normal','high','normal','high'],
    'windy' : ['false','true','false','false','false','true','true','false','false','false','true','true','false','true'],
    'play' : ['no','no','yes','yes','yes','no','yes','no','yes','yes','yes','yes','yes','no']
}

In [3]:
df = pd.DataFrame(data)

In [5]:
def calc_avg(df, feature):
    variables = pd.unique(df[feature])
    target_values = pd.unique(df['play'])
    avg = 0
    for var in variables:
        entropy = 0
        for target in target_values:
            x = len(df[df[feature] == var][df['play'] == target][feature])
            y = len(df[df[feature] == var][feature])
            e = x / (y + np.finfo(float).eps)
            entropy += -e * np.log2(e + np.finfo(float).eps)
        attr = y / len(df)
        avg += attr * entropy
    return avg

def calc_entropy():
    values = pd.unique(df['play'])
    entropy = 0
    n = len(df)
    for value in values:
        x = df['play'].value_counts()[value] / n
        entropy += -x * np.log2(x)
    return entropy

def find_node(df):
    entropy_attrs = {}
    entropy = calc_entropy()
    for i in range(len(df.columns) - 1):
        entropy_attrs[df.columns[i]] = calc_avg(df, df.columns[i])
    
    gain = {}
    for key in entropy_attrs:
        gain[key] = entropy - entropy_attrs[key]
        
    node = max(gain, key=gain.get)
    return node

def dropData(df, node, value):
    return df[df[node] == value].reset_index(drop=True)

def buildTree(df, tree=None):
    node = find_node(df)
    
    if tree is None:
        tree = {}
        tree[node] = {}
        
    values = df[node].unique()
    
    for val in values:
        sub_df = dropData(df, node, val)
        target, count = np.unique(sub_df['play'], return_counts = True)
        #target = array['no','yes'], count = array([2,3])
        #target = array[yes'], count = array([4])
        if len(target) == 1:
            tree[node][val] = target[0]
        else:
            tree[node][val] = buildTree(sub_df)
    
    return tree


In [6]:
buildTree(df)

<ipython-input-5-471a1d2ccbfc>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  x = len(df[df[feature] == var][df['play'] == target][feature])


{'outlook': {'sunny': {'humidity': {'high': 'no', 'normal': 'yes'}},
  'overcast': 'yes',
  'rainy': {'windy': {'false': 'yes', 'true': 'no'}}}}